# What every quant should know about Numerical Computing

## _Or: A practioners guide to Numerical Analysis_

## Agenda

* IEEE FLoating Point representation
* Special forms
* Rounding, FMA
* Overflow, Bignums
* SIMD/AVX2


## 1.0 Floating Point numbers

|Significand	|Exponent	|Scientific notation	|Fixed-point value|
|----|----|----|----|
|1.5	|4	|1.5 ⋅ 10<sup>4</sup>	|15000|
|-2.001	|2	|-2.001 ⋅ 10<sup>2</sup>	|-200.1|
|5	|-3	|5 ⋅ 10<sup>-3</sup>	|0.005|
|6.667	|-11	|6.667 ⋅ 10<sup>-11</sup>	|0.00000000006667|

### Binary Floating Point

![Double Precision IEEE](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/IEEE_754_Double_Floating_Point_Format.svg/618px-IEEE_754_Double_Floating_Point_Format.svg.png)

### The Sign bit

In [1]:
bits(1.0)

"0011111111110000000000000000000000000000000000000000000000000000"

In [2]:
bits(-1.0)

"1011111111110000000000000000000000000000000000000000000000000000"

In [141]:
function floatbits(x::Float64)
    b = bits(x)
    b[1:1]*"|"*b[2:12]*"|"*b[13:end]
end

floatbits (generic function with 1 method)

In [90]:
floatbits(1.0)

"0|01111111111|0000000000000000000000000000000000000000000000000000"

### The exponent (powers of 2)

In [90]:
floatbits(1.0)

"0|01111111111|0000000000000000000000000000000000000000000000000000"

In [3]:
Int(0b01111111111)

1023

### Significand

The Signifcand stored as 52 bits, and is interpreted as <b> 1.b<sub>1</sub>b<sub>2</sub>&#x2026;b<sub>52</sub> </b>

In [91]:
floatbits(1.0)

"0|01111111111|0000000000000000000000000000000000000000000000000000"

In [5]:
1 * 2^(1023-1023)

1

In [92]:
floatbits(1.5)

"0|01111111111|1000000000000000000000000000000000000000000000000000"

In [7]:
(1 + 1/2) * 2 ^ (1023-1023)

1.5

In [9]:
frexp(1.5)

(0.75, 1)

In [93]:
floatbits(1.75)

"0|01111111111|1100000000000000000000000000000000000000000000000000"

In [11]:
(1 + 1/2 + 1/2^2) * 2 ^ (1023-1023)

1.75

In [94]:
floatbits(15.0)

"0|10000000010|1110000000000000000000000000000000000000000000000000"

In [13]:
Int(0b10000000010)

1026

In [14]:
(1 + 1/2 + 1/4 + 1/8)

1.875

In [15]:
1.875 * 2^(1026-1023)

15.0

In [16]:
frexp(15.0)

(0.9375, 4)

In [17]:
ldexp(0.9375, 4)

15.0

#### Approximate representation

Not all decimal numbers are exactly representable as binary floating point number. And many decimal values can be approximated by the same float. 

In [88]:
0.1 > 1//10

true

In [95]:
floatbits(0.1)

"0|01111111011|1001100110011001100110011001100110011001100110011010"

In [19]:
Rational(0.1)

3602879701896397//36028797018963968

In [20]:
float(big(Rational(0.1)))

1.000000000000000055511151231257827021181583404541015625000000000000000000000000e-01

In [96]:
floatbits(0.10000000000000001)

"0|01111111011|1001100110011001100110011001100110011001100110011010"

In [22]:
bits(0.10000000000000001) == bits(0.1)

true

In [23]:
eps(0.1)

1.3877787807814457e-17

In [24]:
reinterpret(Float64, 0b0011111110111001100110011001100110011001100110011001100110011011) - 

reinterpret(Float64, 0b0011111110111001100110011001100110011001100110011001100110011010)


1.3877787807814457e-17

In [25]:
nextfloat(0.1)

0.10000000000000002

In [97]:
floatbits(nextfloat(0.1))

"0|01111111011|1001100110011001100110011001100110011001100110011011"

In [98]:
floatbits(0.1)

"0|01111111011|1001100110011001100110011001100110011001100110011010"

## 2.0 Special Forms

### Signed Zeros

In [142]:
floatbits(0.0)

"0|00000000000|0000000000000000000000000000000000000000000000000000"

In [143]:
floatbits(-0.0)

"1|00000000000|0000000000000000000000000000000000000000000000000000"

In [144]:
0.0 == -0.0

true

In [145]:
0.0 === -0.0

false

### Infinity

An exponent of all 1<sub>s</sub>, and signicand of all 0<sub>s</sub> is Infinity

In [146]:
floatbits(Inf)

"0|11111111111|0000000000000000000000000000000000000000000000000000"

In [147]:
floatbits(-Inf)

"1|11111111111|0000000000000000000000000000000000000000000000000000"

### Not A Number (NaN)

An exponent of all 1<sub>s</sub>, and a non zero significand is NaN

In [148]:
floatbits(NaN)

"0|11111111111|1000000000000000000000000000000000000000000000000000"

In [149]:
reinterpret(Float64, 0b0111111111110000000000000000000000000000000000000000000000000001)

NaN

In [35]:
NaN == NaN

false

In [163]:
NaN === NaN

true

In [37]:
0/0

NaN

In [38]:
0/0 == 0/0

false

In [39]:
1.5/0

Inf

### Subnormal

In [40]:
typemin(Float64)

-Inf

In [41]:
reinterpret(Float64, 0b0000000000010000000000000000000000000000000000000000000000000000)

2.2250738585072014e-308

In [42]:
2.0^(1-1023)

2.2250738585072014e-308

In [43]:
 reinterpret(Float64, 0b0000000000001000000000000000000000000000000000000000000000000000) 

1.1125369292536007e-308

In [44]:
reinterpret(Float64, 0b0000000000000000000000000000000000000000000000000000000000000001)

5.0e-324

Subnormals are useful for gradual underflow. With subnormals, addition or subtraction of normal floats will *not* underflow. This prevents erroneous division by zero errors. 

In [45]:
3.001e-308 - 3e-308 

1.0e-311

In [46]:
issubnormal(1.0)

false

In [47]:
issubnormal(3.001e-308 - 3e-308 )

true

In [48]:
set_zero_subnormals(true)

true

In [49]:
3.001e-308 - 3e-308 

0.0

Subnormals can sometimes have a performance impact

In [50]:
using BenchmarkTools

function timestep(b::Vector{T}, a::Vector{T}, Δt::T) where T
    @assert length(a)==length(b)
    n = length(b)
    b[1] = 1                            # Boundary condition
    for i=2:n-1
        b[i] = a[i] + (a[i-1] - T(2)*a[i] + a[i+1]) * Δt
    end
    b[n] = 0                            # Boundary condition
end

function heatflow(a::Vector{T}, nstep::Integer) where T
    b = similar(a)
    for t=1:div(nstep,2)                # Assume nstep is even
        timestep(b,a,T(0.1))
        timestep(a,b,T(0.1))
    end
end


a = zeros(Float32,1000);
heatflow(a, 1000) #Force compile

In [51]:
for trial=1:6
    a = zeros(Float32,1000)
    set_zero_subnormals(iseven(trial))  # Odd trials use strict IEEE arithmetic
    @time heatflow(a,1000)
end

  0.002992 seconds (1 allocation: 4.063 KiB)
  0.001782 seconds (1 allocation: 4.063 KiB)
  0.004246 seconds (1 allocation: 4.063 KiB)
  0.001888 seconds (1 allocation: 4.063 KiB)
  0.003259 seconds (1 allocation: 4.063 KiB)
  0.001740 seconds (1 allocation: 4.063 KiB)


## 3.0 Rounding



In [1]:
0.1 + 0.2

0.30000000000000004

In [9]:
big(0.1) + big(0.2)

3.000000000000000166533453693773481063544750213623046875000000000000000000000000e-01

In [8]:
0.1 + 0.2 - (big(0.1) + big(0.2))  < 2 * eps(0.3)

true

In [6]:
eps(0.3)

5.551115123125783e-17

Floating point operations are not associative.

In [64]:
(0.1 + 0.2) + 0.3

0.6000000000000001

In [65]:
0.1 + (0.2 + 0.3)

0.6

In [53]:
sum([1.0, 10e100, 1.0, -10e100])

0.0

In [54]:
1.0 + 10e100 + 1.0 +  -10e100

0.0

In [55]:
sum_kbn([1.0, 10e100, 1.0, -10e100])

2.0

In [56]:
sum_kbn([0.2,0.2,0.2])

0.6000000000000001

### Cancellation

Errors can blow up when subtracting two numbers that are close together. Consider the following function, which can be shown to be:   `f(x) < 0.5 ∀ x`

In [57]:
f(x) = (1 - cos(x))/x^2

f (generic function with 1 method)

In [58]:
f(1.2e-8)

0.7709882115452477

In [59]:
cos(1.2e-8)

0.9999999999999999

In [60]:
1-0.9999999999999999

1.1102230246251565e-16

In [61]:
1.1102230246251565e-16 / 1.44e-16

0.7709882115452477

In [38]:
f2(x) = 0.5 * (sin(x/2) / (x/2))^2

f2 (generic function with 1 method)

In [40]:
f2(1.2e-8)

0.5

### Special Functions

In [29]:
sin(π)

1.2246467991473532e-16

In [36]:
sin(1000_000π)

-2.231912181360871e-10

In [37]:
sinpi(1000_000)

0.0

In [35]:
cospi(10000000000000)

1.0

In [34]:
cos(10000000000000pi)

0.9999963627567049

$$
\begin{equation}
\exp(x) = \sum_{n=0}^\infty \frac{x^n}{n!} = 1 + x + \frac12 x^2 + \dots
\end{equation}
$$

Therefore, for $x \ll 1$, we have $\exp(x) \approx 1 + x$

Hence cancellation can occur when calculating $\exp(x) -1$, for $x \ll 1$

In [62]:
x = 1e-8
exp(x) - 1

9.99999993922529e-9

In [61]:
expm(x)

1.00000001

### Rounding Modes

Default rounding mode is `RoundNearest`, which rounds to the nearest representable value, with ties rounded towards the nearest value with an even least significant bit

In [106]:
x = 1.1; y = 0.1;
x+y

1.2000000000000002

In [108]:
setrounding(Float64,RoundDown) do
    x + y
end

1.2

Rounding mode can be used to verify the stability of numerical operations

In [117]:
x=rand(1000_000);

In [118]:
reduce(+, 0.0, x)

499934.62527572236

In [120]:
sum(x)

499934.6252757221

In [121]:
setrounding(Float64,RoundDown) do
   reduce(+, 0.0, x)
end

499934.6252567983

In [123]:
setrounding(Float64,RoundDown) do
   sum(x)
end

499934.62527572043

In [124]:
reduce(+, 0.0, x) - setrounding(Float64,RoundDown) do
   reduce(+, 0.0, x)
end

1.892406726256013e-5

In [125]:
sum(x) - setrounding(Float64,RoundDown) do
   sum(x)
end

1.6880221664905548e-9

## 4.0 Fused Multiply Add

In [64]:
fma(3.0, 4.0, 5.0)

17.0

In [103]:
3.0 * 4.0 + 5

17.0

In [63]:
@code_native fma(3.0, 4.0, 5.0)

	.section	__TEXT,__text,regular,pure_instructions
Filename: floatfuncs.jl
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 233
	vfmadd213sd	%xmm2, %xmm1, %xmm0
	popq	%rbp
	retq
	nopl	(%rax,%rax)


In [91]:
quad(b, a, c) = sqrt(b^2 - 4*a*c)

quad (generic function with 1 method)

In [92]:
quadf(b, a, c) = sqrt(fma(b, b, -4*a*c))

quadf (generic function with 1 method)

In [94]:
quad(.2, .1, .1)

0.0

In [95]:
quadf(.2, .1, .1)

LoadError: DomainError:

In [98]:
fma(.2, .2, -4*.1*.1)

-3.3306690738754695e-18

In [100]:
0.2^2

0.04000000000000001

In [101]:
-4 * .1 * .1

-0.04000000000000001

`muladd` can sometime be faster. 

In [63]:
muladd(3, 4, 5)

17

### 5.0 Overflow

In [66]:
typemax(Int64)

9223372036854775807

In [67]:
9223372036854775807 + 1

-9223372036854775808

In [69]:
bits(9223372036854775807)

"0111111111111111111111111111111111111111111111111111111111111111"

In [150]:
bits(9223372036854775807 + 1)

"1000000000000000000000000000000000000000000000000000000000000000"

In [71]:
typemax(Float64)

Inf

In [153]:
a=reinterpret(Float64, 0b0111111111101111111111111111111111111111111111111111111111111111)

1.7976931348623157e308

In [154]:
floatbits(a)

"0|11111111110|1111111111111111111111111111111111111111111111111111"

In [86]:
a+1

1.7976931348623157e308

### Bignums

In [16]:
a=big(typemax(Int64))

9223372036854775807

In [17]:
a+1

9223372036854775808

In [126]:
big(0.1)

1.000000000000000055511151231257827021181583404541015625000000000000000000000000e-01

In [127]:
big(0.1) == 0.1

true

In [129]:
parse(BigFloat, "0.1")

1.000000000000000000000000000000000000000000000000000000000000000000000000000002e-01

In [131]:
parse(BigFloat, "0.1") == 0.1

false

In [132]:
2.0^66 / 3

2.4595658764946067e19

In [133]:
BigFloat(2.0^66) / 3

2.459565876494606882133333333333333333333333333333333333333333333333333333333344e+19

In [139]:
factorial(BigInt(40))

815915283247897734345611269596115894272000000000

## 6.0 SIMD

##### 128 (SSE), 256 (AVX) or 512 (AVX512) bit parallel operations

In [155]:
function mysum(a::Vector)
    total = zero(eltype(a))
    @simd for x in a
        total += x
    end
    return total
end

mysum (generic function with 1 method)

![Serial Sum](https://juliacomputing.com/assets/img/new/auto-vectorization1.png)

![Simd Sum](https://juliacomputing.com/assets/img/new/auto-vectorization2.png)

  * (V)ADDPS: Takes two 128/256/512 bit values and adds 4/8/16 single precision values in parallel
  * (V)ADDPD: Takes two 128/256/512 bit values and adds 2/4/8 double precision values in parallel
  * (V)PADD(B/W/D/Q): Takes two 128/256/512 bit values and adds (up to 64) 8/16/32/64-bit integers in parallel
  * (V)ADDSUBP(S,W): Takes two inputs, the operation is (+,-,+,-,…) on packed values
  * There are also a few more exotic instructions that involve horizontal adds, saturating, etc.

In [13]:
@code_native mysum(rand(Float64 , 100000)) ;

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[10]
Source line: 67
	movq	8(%rdi), %rax
Source line: 64
	movq	24(%rdi), %rdx
	xorl	%ecx, %ecx
Source line: 79
	testq	%rdx, %rdx
	cmovnsq	%rdx, %rcx
Source line: 68
	testq	%rax, %rax
	jle	L258
Source line: 79
	leaq	-1(%rcx), %rsi
	cmpq	%rdx, %rsi
	jae	L266
Source line: 50
	movq	(%rdi), %r9
Source line: 66
	leaq	96(%r9), %r8
	movq	%rax, %r10
	andq	$-16, %r10
	vxorpd	%xmm0, %xmm0, %xmm0
	xorl	%edi, %edi
Source line: 50
	vxorpd	%ymm1, %ymm1, %ymm1
Source line: 66
	jmp	L240
	nopw	%cs:(%rax,%rax)
Source line: 71
L80:
	testq	%rax, %rax
	jle	L240
Source line: 50
	cmpq	$16, %rax
	jae	L99
	xorl	%edx, %edx
	jmp	L208
L99:
	movq	%rax, %rdx
	andq	$-16, %rdx
	je	L206
	vblendpd	$1, %ymm0, %ymm1, %ymm0 ## ymm0 = ymm0[0],ymm1[1,2,3]
	vxorpd	%ymm2, %ymm2, %ymm2
Source line: 74
	movq	%r10, %rsi
	movq	%r8, %rcx
	vxorpd	%ymm3, %ymm3, %ymm3
	vxorpd	%ymm4, %ymm4, %ymm4
	nopw	%cs:(%rax,%rax)
Source line: 4
L144:
	vaddpd	-96(%rcx), %ymm0, %ymm0
	vad

In [157]:
function mysum_basic(a::Vector)
    total = zero(eltype(a))
    for x in a
        total += x
    end
    return total
end

mysum_basic (generic function with 1 method)

In [160]:
@code_native mysum_basic(rand(Float64 , 100000)) ;

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[157]
Source line: 3
	movq	8(%rdi), %rax
	vxorpd	%xmm0, %xmm0, %xmm0
	testq	%rax, %rax
	je	L51
	movq	(%rdi), %rdx
	movq	24(%rdi), %rsi
	vxorpd	%xmm0, %xmm0, %xmm0
	xorl	%ecx, %ecx
	nopw	(%rax,%rax)
L32:
	cmpq	%rsi, %rcx
	jae	L52
Source line: 4
	vaddsd	(%rdx,%rcx,8), %xmm0, %xmm0
Source line: 3
	addq	$1, %rcx
	cmpq	%rcx, %rax
	jne	L32
Source line: 6
L51:
	retq
L52:
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 3
	movq	%rsp, %rax
	leaq	-16(%rax), %rsi
	movq	%rsi, %rsp
	addq	$1, %rcx
	movq	%rcx, -16(%rax)
	movabsq	$jl_bounds_error_ints, %rax
	movl	$1, %edx
	callq	*%rax
	nopl	(%rax,%rax)


In [162]:
using BenchmarkTools
a=rand(1000_000)
@btime mysum(a)
@btime mysum_basic(a)

  461.743 μs (1 allocation: 16 bytes)
  1.078 ms (1 allocation: 16 bytes)


499926.86464234843

## 7.0 Further Reading

  * IEEE Standard for Floating-Point Arithmetic
    * https://ieeexplore.ieee.org/document/4610935/
  * The Accuracy and Stability of Numerical Algorithms
    * Nicholas J Higham (2<sup>nd</sup> Edition, 2002)